In [14]:
import pymysql.cursors
import pandas as pd
import numpy as np
import re
import math

In [2]:
def connect_to_database(db="UltimateGuitarTabs"):
    """
    Connects to MySQL database and 
    returns the connection"""
    db = pymysql.connect(host="localhost",  # your host 
                         user="root",       # username
                         passwd="",     # password
                         db=db)   # name of the database
    return db

def print_table(tableName):
    """
    Prints database table
    """
    db = connect_to_database()
    cur = db.cursor()

    # Select data from table using SQL query.
    cur.execute("SELECT * FROM "+tableName)
    # print the first and second columns      
    for row in cur.fetchall() :
        print(row)
        
def get_table(tableName):
    db = connect_to_database()
    cur = db.cursor()

    # Select data from table using SQL query.
    cur.execute("SELECT * FROM "+tableName)
    
    return(cur.fetchall())


In [3]:
Chords = pd.DataFrame(list(get_table('Chords')), columns=['Song', 'Artist', 'Key', 'Capo', 'Chords'])
Chords

,Song,Artist,Key,Capo,Chords
0,Hallelujah,Jeff Buckley,Db,1,"C,Am,C,Am,C,Am,C,Am,F,G,C,G,C,F,G,Am,F,G,E7,Am..."
1,Im Yours,Jason Mraz,G,4,"G,D,Em,C,G,D,Em,C,G,D,Em,C,G,D,Em,C,G,D,Em,C,G..."
2,Wonderwall,Oasis,F#m,2,"Em7,G,Dsus4,A7sus4,Em7,G,Dsus4,A7sus4,Em7,G,Ds..."
3,Wish You Were Here,Pink Floyd,Em,0,"Em7,G,Em7,G,Em7,A7sus4,Em7,A7sus4,G,C,D/F#,G,D..."
4,Hey Soul Sister,Train,C,4,"C,G,Am,F,C,G,Am,F,C,G,Am,F,G,C,G,Am,F,C,G,Am,F..."
5,Wonderwall,Oasis,D,0,"F#m,A,Esus4,B,F#m,A,Esus4,B,F#m,A,Esus4,B,F#m,..."
6,The Only Exception,Paramore,B,4,"G,Dm,Cmaj7,G,Dm,Cmaj7,G,Dm,Cmaj7,G,Dm,Cmaj7,G,..."
7,Love Story,Taylor Swift,C,2,"C,G,Am,F,C,F,Am,F,C,F,Am,G,F,G,Am,C,F,G,Am,F,G..."
8,Creep,Radiohead,G,0,"G,B,C,Cm,G,B,C,Cm,G,B,C,Cm,G,B,C,Cm,G,B,C,Cm,G..."
9,Iris,Goo Goo Dolls,Bm,0,"Bm,Bsus2,G,D,Em,G,Bm,A,G,D,Em,G,Bm,A,G,D,Em,G,..."


In [27]:
def fix_accidental(note, accidental):
    notes = np.asarray(['A', 'B', 'C', 'D', 'E', 'F', 'G'])

    note_idx = int(np.where(notes == note)[0])
    if accidental == '#':
        # Account for B# or E#
        if note == 'B':
            return('C', '')
        elif note == 'E':
            return('F', '')
        return(note,accidental)
    elif accidental == 'b':
        # Account for Cb or Fb
        if note == 'C':
            return('B', '')
        elif note == 'F':
            return('E', '')
        return(notes[note_idx - 1],'#')
    else:
        return(note, accidental)
            
        
def clean_chords(chords):
    """
    This function takes in a comma-separated string of 
    chords and cleans it by removing any base note variations, or
    other chord embelishments.  Diminished labels are kept as these
    are used in the chord progression table. The purpose of this
    is to clean the chords to match the labels within the chord
    progression table.
    
    returns:
        new_chords - array of newly cleaned chords to be tabulated
                        by the chord progression table
    """
    
    # Pattern grouping: 1=(chord pitch) 2=(base note) 3=(chord type) 4=(base note)
    pattern = "^([A-G]+)(\/[A-G]*[b#])*([(?m)|(?m\d)|(?b\d)|(?#\d)|(?maj\d)|\
    (?add\d)|(?sus\d)|(?aug)|(?aug\d)|(?dim)|(?dim\d)]*)(\/[A-G]*[b#])*"        
    prog = re.compile(pattern)

    pattern2 = "^([A-G])([b#])?(m$|m\d$)?(dim$|dim\d$)?"
    prog2 = re.compile(pattern2)

    chords = chords.split(',')
    new_chords = [""]*len(chords)
    for i in range(len(chords)):
        curr_chord = chords[i]
        groups = prog.findall(curr_chord)[0] 
        no_base = groups[0] + groups[2]
        no_num = re.sub(pattern="\d", repl="", string=no_base)

        groups = prog2.findall(no_num)[0]
        note,accidental = fix_accidental(groups[0], groups[1])
        new_chords[i] = note + accidental + groups[2] + groups[3]
        
    return(new_chords)


def get_key_tbls():
    """
    Helper function that reads in a chord progression table
    and creates a dictionary that maps chord names to their
    indices on the chord progression table. This dictionary
    will be used to tabulate a 'key table' to determine the
    key of a song.
    
    returns:
        Key_dict - Dictionary mapping chord names to indices
        Keys - array of keys that correspond to the order
                of the progression table
    """
    Key_tbl = pd.read_csv('../data/key_table.csv')
    Keys = list(Key_tbl.key)
    Tbl = np.asmatrix(Key_tbl.iloc[:,1:8])

    # Storing all possible chords 
    all_chords = []
    for i in range(Tbl.shape[0]):
        for j in np.asarray(Tbl[i])[0]:
            all_chords.append(j)
    all_chords = np.unique(all_chords)
    
    # Creating dict(key='chord', val='indices in progression tbl')
    Key_dict = {}
    for chord in all_chords:
        Key_dict[chord] = np.where(Tbl == chord)

    return(Key_dict, Keys)

def is_rel_min(comp_key, act_key):
    """
    Determines if the actual key marked indicated
    UltimateGuitarTabs.com is the relative minor
    of the actual key (i.e., Am(rel. minor) and C(actual))
    
    args:
        comp_key - Computed key
        act_key - UltimateGuitarTabs.com key
    returns:
        - True if it's relative minor, False otherwise
    """
    notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
    if str(act_key[-1]) != 'm': #If not minor, return 
        return(False)
    if len(comp_key) == 2:
        note, accidental = fix_accidental(comp_key[0],comp_key[1])
        comp_key = note + accidental
    comp_idx = np.where(np.asarray(notes) == comp_key)[0][0]
    rel_min_idx = comp_idx - 3
    if notes[rel_min_idx] + 'm' == act_key:
        return(True)

def compute_key(Key_dict, Keys, chords):
    """
    Computes the key of a song by analyzing
    the chords used within the song. A theoretical
    key matrix where each row specifies the types of chords
    that coincide with a key is used to tabulate
    the existing chords. The largest row sum is the
    computed key of the song. 
    
    args:
        Key_dict - Python dictionary containing all possible
                   chords and their indexes in the theoretical key matrix.
                   This is used to tabulate a matrix of zeros.
        Keys - List of possible keys corresponding to theoretical key matrix
        chords - String of chords separated by commas
                   
    returns:
        - The computed key
        - List of cleaned chords, only major or minor
    """
    chords = clean_chords(chords)
    count_mat = np.zeros((12,7)) # Matrix of zeros to tabulate chord occurences
    start_chord = chords[0]
    # Tabulating chords
    for chord in chords:
        count_mat[Key_dict[chord]] += 1
        
    computed_key = Keys[np.argmax(np.sum(count_mat, axis = 1))]
    return(computed_key, chords)
    
def transpose_C(Key_dict, Keys, chords, states):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    comp_key, chords = compute_key(Key_dict, Keys, chords)
    
    new_chords = [""]*len(chords) # Empty list for new chords
    if comp_key == 'C':
        for chord in chords:
            states.add(chord)
        return(states, \
               {'orig_key': comp_key, \
                'trans_chords': chords})
    else:
        steps = np.where(np.asarray(notes) == comp_key)[0][0]
        for i in range(len(chords)):
            is_min = False
            curr_chord = chords[i]
            if curr_chord[-1] == 'm':
                is_min = True
                curr_chord = curr_chord[0:-1]
            chord_idx = np.where(np.asarray(notes) == curr_chord)[0][0]
            new_chord_idx = chord_idx - steps
            new_chords[i] = notes[new_chord_idx] + 'm' if is_min else notes[new_chord_idx]
            states.add(new_chords[i])
        return(states, \
               {'orig_key': comp_key, \
                'trans_chords': new_chords})
def get_songs():
    Key_dict, Keys = get_key_tbls()

    songs = [""]*Chords.shape[0]
    states = set()
    for i in range(Chords.shape[0]):
        states, songs[i] = transpose_C(Key_dict, Keys, Chords['Chords'][i], states)
    states = sorted(list(states))
    return(states, songs)

def create_transition_mat(states, chords):
    num_states = len(states)
    trans_mat = np.zeros((num_states, num_states))
    for i in range(1, len(chords)):
        curr_chord = chords[i - 1]
        curr_chord_idx = np.where(np.asarray(states) == curr_chord)[0][0]
        
        next_chord = chords[i]
        next_chord_idx = np.where(np.asarray(states) == next_chord)[0][0]
        trans_mat[curr_chord_idx, next_chord_idx] += 1

    row_sums = np.sum(trans_mat, axis = 1)
    trans_mat = trans_mat/row_sums
    return(np.nan_to_num(trans_mat))

def compute_euclidean(mat1, mat2):
    # Computes euclidean distance between two
    # transition matrices
    # NOTE: Will produce runtime warning with division by zero
    mat_dif = mat1 - mat2
    return(math.sqrt(np.sum(np.multiply(mat_dif, mat_dif)))/mat1.shape[0])

def get_similar_songs(states, chords, all_songs):
    tm = create_transition_mat(states,chords)
    dist = [0]*len(all_songs)
    for i in range(len(all_songs)):
        curr_tm = create_transition_mat(states,all_songs[i]['trans_chords'])
        dist[i] = compute_euclidean(tm, curr_tm)
        
    dist_ord = np.argsort(dist) # Sorting by distance ascending
    
    idxs = np.unique(list(Chords['Song'][dist_ord]), return_index=True)[1] # Indexes of unique entries
    # np.unique() automatically sorts, so we must resort by distance
    sim_songs = list(Chords['Song'][dist_ord[sorted(idxs)]])
    return(sim_songs)


In [7]:
states, songs = get_songs()
test_chords = songs[4]['trans_chords']
print(Chords[4:5])

              Song Artist Key  Capo  \
4  Hey Soul Sister  Train   C     4   

                                              Chords  
4  C,G,Am,F,C,G,Am,F,C,G,Am,F,G,C,G,Am,F,C,G,Am,F...  


In [8]:
sim = get_similar_songs(states, test_chords, songs)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:188: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:196: RuntimeWarning: overflow encountered in multiply


In [10]:
sim

['Hey Soul Sister',
 'Here Without You',
 'Open The Eyes Of My Heart',
 'Wonderful Tonight',
 'Sex',
 'Push',
 'Go Your Own Way',
 'Your Call',
 'Youre Beautiful',
 'The Funeral',
 'Down Under',
 'Run',
 'White Horse',
 'Fall For You',
 'Many Of Horror',
 'She Will Be Loved',
 'You Found Me',
 'Almost Lover',
 'Naive',
 'Touch The Sky',
 'Who Am I',
 'The Best Day',
 'Breathe',
 'Your Guardian Angel',
 'Please Please Please Let Me Get What I Want',
 'Right Here Waiting',
 'Numb',
 'Flaws',
 'Forever And Always',
 'Two Is Better Than One',
 'If I Ever Leave This World Alive',
 'Dont Think Twice Its All Right',
 'Somewhere Only We Know',
 'One Of Us',
 'I Want To Break Free',
 'At The Bottom Of Everything',
 'Torn',
 'There Is A Light That Never Goes Out',
 'For The First Time',
 'Can You Feel The Love Tonight',
 'Dreams',
 'King And Lionheart',
 'All I Want',
 'Im Yours',
 'Thats What You Get',
 'Glycerine',
 'Let It Be',
 'Over You',
 'Girls Just Want To Have Fun',
 'Here I Go Again',


['C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'C',
 'G',
 'Am',
 'F',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'F',
 'G',
 'C',
 'G',
 'F',
 'G',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'C',
 'G',
 'Am',
 'F',
 'G',
 'C']

Sudo code:
- Read in each string of chords and clean them
- Tabulate # of times each chord comes up in a given song
 1. Take note of each unique chord
 2. Call np.where for each and store the corresponding indices
     - This will save time so we don't call np.where for every single chord of every song
- Row sum and treat the largest row as the key of the song

In [197]:
a=list(combinations_with_replacement(states, 2))
for i in range(len(a)):
    a[i] = a[i][0] + ',' + a[i][1]


In [204]:
np.where(np.asarray(states) == 'B')[0][0]

4

In [200]:
a

['A,A',
 'A,A#',
 'A,A#m',
 'A,Am',
 'A,B',
 'A,Bm',
 'A,C',
 'A,C#',
 'A,C#m',
 'A,Cm',
 'A,D',
 'A,D#',
 'A,D#m',
 'A,Dm',
 'A,E',
 'A,Em',
 'A,F',
 'A,F#',
 'A,F#m',
 'A,Fm',
 'A,G',
 'A,G#',
 'A,G#m',
 'A,Gm',
 'A#,A#',
 'A#,A#m',
 'A#,Am',
 'A#,B',
 'A#,Bm',
 'A#,C',
 'A#,C#',
 'A#,C#m',
 'A#,Cm',
 'A#,D',
 'A#,D#',
 'A#,D#m',
 'A#,Dm',
 'A#,E',
 'A#,Em',
 'A#,F',
 'A#,F#',
 'A#,F#m',
 'A#,Fm',
 'A#,G',
 'A#,G#',
 'A#,G#m',
 'A#,Gm',
 'A#m,A#m',
 'A#m,Am',
 'A#m,B',
 'A#m,Bm',
 'A#m,C',
 'A#m,C#',
 'A#m,C#m',
 'A#m,Cm',
 'A#m,D',
 'A#m,D#',
 'A#m,D#m',
 'A#m,Dm',
 'A#m,E',
 'A#m,Em',
 'A#m,F',
 'A#m,F#',
 'A#m,F#m',
 'A#m,Fm',
 'A#m,G',
 'A#m,G#',
 'A#m,G#m',
 'A#m,Gm',
 'Am,Am',
 'Am,B',
 'Am,Bm',
 'Am,C',
 'Am,C#',
 'Am,C#m',
 'Am,Cm',
 'Am,D',
 'Am,D#',
 'Am,D#m',
 'Am,Dm',
 'Am,E',
 'Am,Em',
 'Am,F',
 'Am,F#',
 'Am,F#m',
 'Am,Fm',
 'Am,G',
 'Am,G#',
 'Am,G#m',
 'Am,Gm',
 'B,B',
 'B,Bm',
 'B,C',
 'B,C#',
 'B,C#m',
 'B,Cm',
 'B,D',
 'B,D#',
 'B,D#m',
 'B,Dm',
 'B,E',
 'B,Em',
 '

In [216]:
tr = np.asmatrix([[0, 1, 1, 0],
             [1, 4, 1, 0]])

In [217]:
col_sums = np.sum(tr, axis = 0)

In [218]:
tr / col_sums

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


matrix([[0. , 0.2, 0.5, nan],
        [1. , 0.8, 0.5, nan]])

In [16]:
math.sqrt(0.025)/2

0.07905694150420949

In [19]:
a = np.asmatrix([[0,1],
                 [0.25, 0.75]])
b = np.asmatrix([[0.1, 0.9],
                 [0.3, 0.7]])
compute_euclidean(a, b)

0.0125

In [26]:
np.sum(np.multiply(a-b, a-b))

0.025

In [25]:
np.sum(a-b)

2.7755575615628914e-17